# AFLOW-SYM

##### Requirements for this Jupyter notebook:
- AFLOW binary (v3.1.225 and higher) in your path
- xz compression (included in steps for AFLOW installation)
- Python's libraries (used in examples):
    - json
    - subprocess
    - os
    - pprint
    - numpy

### Command line usage

#### Generate MgO rocksalt structure

In [ ]:
! aflow --proto=AB_cF8_225_a_b:Mg:O --params=3.5 > MgO.poscar

#### Calculate the space group number and standard label

In [ ]:
! aflow --aflowSG < MgO.poscar

#### Calculate the Wyckoff positions
Wyckoff positions and cell choices are consistent with the International Tables for Crystallography (ITC)

![ITC_standard_Wyckoff](images/ITC_standard_Wyckoff.png)

In [ ]:
! aflow --wyccar < MgO.poscar

Control the space group setting/cell choice with `--setting=<number>`.
Different cell/setting choice exist for: 
- centrosymmetric space groups with two origin choices (24 space groups, *e.g.*, SG=#227):  
    - `--setting=1` : origin centered on high site symmetry (not inversion site) 
    - `--setting=2` : origin centered on inversion site 
- rhombohedral space groups (*e.g.*, SG=#166):  
    - `--setting=1` : rhombohedral setting  
    - `--setting=2` : hexagonal setting 
- monoclinic space groups (*e.g.*, SG=#5):  
    - `--setting=1` : unique axis-b  
    - `--setting=2` : unique axis-c 

In [ ]:
! aflow --proto=ABC3_hR10_167_a_b_e-002:C:Ca:O | aflow --wyccar --setting=1   # calcite, 1=rhombohedral setting

In [ ]:
! aflow --proto=ABC3_hR10_167_a_b_e-002:C:Ca:O | aflow --wyccar --setting=2   # calcite, 2=hexagonal setting

#### Calculate all space group information (space group number/labels, Wyckoff positions, equations for general Wyckoff position, etc.)

In [ ]:
! aflow --sgdata < MgO.poscar

#### Determine the standard primitive cell (AFLOW Standard)

In [ ]:
! aflow --sprim < MgO.poscar

#### Determine the standard conventional cell (AFLOW Standard)

In [ ]:
! aflow --sconv < MgO.poscar

#### Calculate all symmetry descriptions (lattice, crystal, reciprocal lattice, superlattice)

In [ ]:
! aflow --edata < MgO.poscar

#### Calculate explicity symmetry operations for the point groups (lattice, crystal, site), factor group, and space group

This will return files corresponding to the following symmetry analyses:

- Lattice point group : `aflow.pgroup.out`
- Crystallographic point group : `aflow.pgroup_xtal.out`
- Factor group (unit cell) : `aflow.fgroup.out`
- Reciprocal lattice point group : `aflow.pgroupk.out`
- Dual of crystallographic point group : `aflow.pgroupk_xtal.out`
- Site point group (atom point group) : `aflow_agroup.out`
- Space group : `aflow.sgroup.out`

In [ ]:
! aflow --aflowSYM < MgO.poscar

Notice that symmetry files were written in this directory, containing a comprehensive set of symmetry representations

In [ ]:
! ls

Examine factor group (unit cell) operations : `aflow.fgroup.out` 

In [ ]:
! xzcat aflow.fgroup.out.xz

#### Adjust the tolerance for the space group analysis `--aflowSG=<tolerance>`
Two preset tolerance values are available:
- `tight` : (minimum interatomic distance)/100 (default)
- `loose` : (minimum interatomic distance)/10

A number (in Angstroms) can also be specified.

##### Example

In [ ]:
! aflow --proto=A12BC4_cP34_195_2j_ab_2e-001:P:Pr:Ru | aflow --aflowSG=tight

In [ ]:
! aflow --proto=A12BC4_cP34_195_2j_ab_2e-001:P:Pr:Ru | aflow --aflowSG=0.005

In [ ]:
! aflow --proto=A12BC4_cP34_195_2j_ab_2e-001:P:Pr:Ru | aflow --aflowSG=0.001

Space groups #195, #200, and #204 are all reasonable classifications via the symmetry subgroup relationships and are consistent with the literature for this prototype (see comments in [aflow.org/CrystalDatabase/A12BC4_cP34_195_2j_ab_2e](http://aflow.org/CrystalDatabase/A12BC4_cP34_195_2j_ab_2e)). 

The tolerance values can be adjusted for
- `--aflowSG`   : space group number/label
- `--sgdata`    : full set of space group data
- `--edata`     : extended crystallographic data
- `--wyccar`    : Wyckoff positions
- `--aflowSYM`  : explicit symmetry operations

#### Crystal-spin symmetry analysis
Spin degree of freedom can break symmetry

![crystal_spin_symmetry_example](images/crystal_spin_symmetry_example.png)

Note: not the same as magnetic group symmetry (time reversal = spin flip) 

Add the `--magmom` flag to the command to specify the magnetic moment on each site (same order as geometry file).
AFLOW autodetects collinear or non-collinear based on input size.

The magnetic moment can be specified explicitly via:
- `--magmom=m1,m2,m3,...` : collinear
- `--magmom=mx1,my1,mz1,mx2,...` : non-collinear

or a VASP INCAR or OUTCAR can be read:
- `--magmom=<PATH_TO_INCAR>` : specify path to INCAR file
- `--magmom=<PATH_TO_OUTCAR>` : specify path to OUTCAR file

##### Example
Create an enlarged cell (we will use the standard conventional cell)

In [ ]:
! cat MgO.poscar | aflow --sconv > MgO_sconv.poscar   # first save the structure (use later)
! cat MgO_sconv.poscar

Space group (no magnetic information)

In [ ]:
! cat MgO_sconv.poscar | aflow --aflowSG

Space group (magnetic information)

In [ ]:
! cat MgO_sconv.poscar | aflow --aflowSG --magmom=1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0

Crystal-spin analysis works for
- `--aflowSG`   : space group number/label
- `--sgdata`    : full set of space group data
- `--edata`     : extended crystallographic data
- `--wyccar`    : Wyckoff positions
- `--aflowSYM`  : explicit symmetry operations

#### Calculate high-symmetry $k$-paths in Brillouin zones (AFLOW Standard)
![BZ_kpaths](images/BZ_kpaths.png)

In [ ]:
! aflow --kpath < MgO.poscar

#### A comprehensive README for the AFLOW-SYM functionality is available via:

In [ ]:
! aflow --readme=symmetry

### Python module

To facilitate integration in user workflows, a Python module is available: 

In [ ]:
import json
import subprocess
import os


class Symmetry:

    def __init__(self, aflow_executable='aflow'):
        self.aflow_executable = aflow_executable


    def aflow_command(self, cmd):
        try:
            return subprocess.check_output(
                self.aflow_executable + cmd,
                shell=True
            )
        except subprocess.CalledProcessError:
            print('Error aflow executable not found at: ' + self.aflow_executable)


    def get_symmetry(self, input_file, tol=None, magmoms=None):
        fpath = os.path.realpath(input_file.name)
        command = ' --aflowSYM'
        output = ''


        if tol:
            command += '=' + str(tol)
        if magmoms:
            command += ' --magmom=' + magmoms

        output = self.aflow_command(
            command + ' --print=json --screen_only' + ' < ' + fpath
        )
        res_json = json.loads(output)
        return res_json


    def get_edata(self, input_file, tol=None, magmoms=None):
        fpath = os.path.realpath(input_file.name)
        command = ' --edata'
        output = ''


        if tol:
            command += '=' + str(tol)
        if magmoms:
            command += ' --magmom=' + magmoms

        output = self.aflow_command(
            command + ' --print=json' + ' < ' + fpath
        )
        res_json = json.loads(output)
        return res_json
    
    def get_sgdata(self, input_file, tol=None, magmoms=None):
        fpath = os.path.realpath(input_file.name)
        command = ' --sgdata'
        output = ''


        if tol:
            command += '=' + str(tol)
        if magmoms:
            command += ' --magmom=' + magmoms

        output = self.aflow_command(
            command + ' --print=json' + ' < ' + fpath
        )
        res_json = json.loads(output)
        return res_json

### Calculate full set of symmetry descriptions `Symmetry.get_edata`

In [ ]:
#from aflow_sym import Symmetry                 # import Symmetry class via aflow_sym module
from pprint import pprint                       # import pretty print (pprint)

with open('MgO.poscar', 'r') as input_file:     # open geometry file
    sym = Symmetry(aflow_executable='aflow')    # specify path to AFLOW binary
    output = sym.get_edata(input_file)          # perform extended crystallographic (--edata) symmetry analysis
    pprint(output)                              # pretty print output

#### Pearson symbol:

In [ ]:
pprint(output['Pearson_symbol'])

#### Space group number, Hermann-Mauguin symbol, Schoenflies symbol

In [ ]:
pprint([output['space_group_number'],output['space_group_Hermann_Mauguin'],output['space_group_Schoenflies']])

#### Bravais lattice type

In [ ]:
pprint(output['Bravais_lattice_type'])

#### Wyckoff positions (letter, multiplicity, species, position, and site symmetry)

In [ ]:
pprint(output['Wyckoff_positions'])

### Explicit symmetry operations for input cell (`Symmetry.get_symmetry`)

This will return symmetry objects corresponding to the following symmetry analyses:

- Lattice point group : `pgroup`
- Crystallographic point group : `pgroup_xtal`
- Factor group (unit cell) : `fgroup`
- Reciprocal lattice point group : `pgroupk`
- Dual of crystallographic point group : `pgroupk_xtal`
- Site point group (atom point group) : `agroup`
- Space group : `sgroup`

In [ ]:
with open('MgO_sconv.poscar', 'r') as input_file:
    sym = Symmetry(aflow_executable='aflow')
    output = sym.get_symmetry(input_file)
    pprint(output)

The JSON response can then be manipulated for use in different workflows. 
##### Example: 

Below, we examine the unit cell symmetry operations (`fgroup`). We apply the fractional rotation matrix (`Uf`) and fractional translation (`ftau`) to the fractional position of the atoms. Note: this example uses Python's `numpy` library.

We compare the original and transformed positions.  The atom mappings from AFLOW-SYM (`basis_atoms_map`) are used to confirm the rotated positions.

First, load the structure as a JSON object (for easy manipulation):

In [ ]:
# convert structure JSON object with AFLOW-SYM module
fpath = os.path.realpath('MgO_sconv.poscar')
_sym = Symmetry(aflow_executable='aflow')
str_output = _sym.aflow_command(' --structure2json' + ' < ' + fpath)
str_json = json.loads(str_output)

In [ ]:
import numpy as np                                                  # import numpy library

for sym_op in output['fgroup']:                                     # get unit cell symmetry operations
    print('Hermann-Mauguin symbol:', sym_op['Hermann_Mauguin'])     # print Hermann-Mauguin symbol
    print('fractional rotation matrix:', sym_op['Uf'])              # print fractional rotation matrix
    print('fractional translation:', sym_op['ftau'])                # print fractional translation
    for i, atom in enumerate(str_json['atoms']):                    # loop over atoms, and apply transformation (matrix and translation) to fractional positions
        print('  atom {0} ({1}) - original: {2} | transformed: {3}'.format(i, atom['name'], atom['position'], np.add(np.dot(sym_op['Uf'], atom['position']),sym_op['ftau'])))        
    print('atom mappings:', sym_op['basis_atoms_map'])              # print atom mappings from AFLOW-SYM
    print('-'*75)                                                   # separator

#### Additional information about AFLOW-SYM can be found in the following article:
- D. Hicks, C. Oses, R. H. Taylor, E. Gossett, G. Gomez, C. Toher, M. J. Mehl, O. Levy, and S. Curtarolo, AFLOW-SYM: Platform for the complete, automatic and self-consistent symmetry analysis of crystals, Acta Cryst. A74, 184-203 (2018). [doi=10.1107/S2053273318003066](https://doi.org/10.1107/S2053273318003066)

### Exercises

1. Use AFLOW-SYM to determine the symmetry properties of the structures that you generated for the prototype A_hR1_166_a (`--sgdata` or `--aflowSG`). How do the choice of parameters affect the symmetry of the structure?

In [ ]:
# your commands/code here


2. Determine the k-paths for one of the A_hR1_166_a structures you generated (`--kpath`).

In [ ]:
# your command here
!

3. Calculate the extended crystallographic data for one of the A_hR1_166_a structures (`--edata`). Use the command line and/or the Python module.

In [ ]:
# your command/code here


4. Calculate the symmetry operations for one of the A_hR1_166_a structures (`--aflowSYM`). Use the command line and/or the Python module.

In [ ]:
# your command/code here


5. Pick a material in the AFLOW database [aflow.org](http://aflow.org) and visualize some of the symmetry elements on the entry page.